In [1]:
import tensorflow as tf
from module.gatedTransformer import GatedTransformer

### Import Dataset

In [2]:
import pickle

file_name = "1_20131027.pkl"
open_file = open(file_name, "rb")
seed_data = pickle.load(open_file)
open_file.close()

data, label = seed_data[1]
print(data.shape)
print(label)

(46601, 5)
[1. 0. 0.]


### Prepare dataset

In [3]:
Hz = 200
duration = 1
size = Hz*duration
batch_size = 32

In [4]:
def create_dataset_window(series, label, size, batch_size=32, shift=1):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(size= size, shift = shift, drop_remainder = True)
    ds = ds.flat_map(lambda w: w.batch(size))
    ds = ds.map(lambda w: (w, label))
    return ds.batch(batch_size)

In [5]:
ds = create_dataset_window(data, label, size, batch_size, int(size/100))
len(list(ds))


726

In [6]:
dataset_list = []
for item in seed_data:
    data, label = item
    ds = create_dataset_window(data, label, size, int(size/100))
    dataset_list.append(ds)

stack_dataset = dataset_list[0]
# for dataset in dataset_list[1:]:
#     stack_dataset = stack_dataset.concatenate(dataset)


In [7]:
DATASET_SIZE = 100
train_size = int(0.9 * DATASET_SIZE)
val_size = int(0.05 * DATASET_SIZE)
test_size = int(0.05 * DATASET_SIZE)

stack_dataset = stack_dataset.shuffle(DATASET_SIZE)
train_dataset = stack_dataset.take(train_size).prefetch(100).repeat()
test_dataset = stack_dataset.skip(train_size).prefetch(100)
val_dataset = test_dataset.take(val_size).prefetch(100)
test_dataset = test_dataset.skip(val_size).prefetch(32)

In [8]:
 num_layers = 8
d_model = 512
num_heads = 8
dff = 1024

target_size = 3
model = GatedTransformer(num_layers, d_model, num_heads, dff, target_size)

In [9]:
loss_object = tf.keras.losses.CategoricalCrossentropy()

In [10]:
def loss(model, x, y, training):
  # training=training is needed only if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
    y_ = model(x, training=training)

    return loss_object(y_true=y, y_pred=y_)


In [11]:
def grad(model, inputs, targets):
    with tf.GradientTape() as tape:
        loss_value = loss(model, inputs, targets, training=True)
    return loss_value, tape.gradient(loss_value, model.trainable_variables)

In [12]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

In [13]:
## Note: Rerunning this cell uses the same model variables

# Keep results for plotting
train_loss_results = []
train_accuracy_results = []

num_epochs = 3

for epoch in range(num_epochs):
    epoch_loss_avg = tf.keras.metrics.Mean()
    epoch_accuracy = tf.keras.metrics.CategoricalAccuracy()

    # Training loop - using batches of 32
    for x, y in train_dataset.take(1):
    # Optimize the model
        loss_value, grads = grad(model, x, y)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        # Track progress
        epoch_loss_avg.update_state(loss_value)  # Add current batch loss
        # Compare predicted label to actual label
        # training=True is needed only if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        epoch_accuracy.update_state(y, model(x, training=True))

        # End epoch
        train_loss_results.append(epoch_loss_avg.result())
        train_accuracy_results.append(epoch_accuracy.result())

   
    print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch,
                                                                epoch_loss_avg.result(),
                                                                epoch_accuracy.result()))

Epoch 000: Loss: 1.155, Accuracy: 100.000%
Epoch 001: Loss: 0.002, Accuracy: 100.000%
Epoch 002: Loss: 0.000, Accuracy: 100.000%
